In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor
import seaborn as sns
sns.set_style("dark")
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

#from warnings import filterwarnings
#filterwarnings('ignore')

data=None
def absolute_correlations(col, df=data):
    #absolute_values = np.abs(df[col])
    corrs = pd.DataFrame(df.select_dtypes(include=[np.number]).corrwith(df[col]), columns=['correlation'])
    corrs['absol'] = np.abs(corrs['correlation'])
    return corrs.sort_values('absol', ascending=False).drop('absol', axis=1).tail(len(corrs)-1)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_json('train.json')

data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

from scipy.stats import mode

In [15]:
x = data['ingredients'].apply(', '.join)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['cuisine'])

In [16]:
param_grid = {
    'tfidfvectorizer__min_df': [1,3],#np.arange(1,11,1),
    'tfidfvectorizer__max_df': np.arange(.08, .09, .005),
    'xgbclassifier__n_estimators': [100],
    'xgbclassifier__max_depth': np.arange(2,3,1)
}

kf = StratifiedKFold(n_splits=3)

pl = make_pipeline(
    TfidfVectorizer(),
    XGBClassifier()
)

grid = GridSearchCV(pl, param_grid, cv=kf, scoring='roc_auc')\
.fit(x, y)

clf = grid.best_estimator_

clf

ValueError: multiclass format is not supported

In [12]:
param_grid = {
    'tfidfvectorizer__min_df': [1,3,5],#np.arange(1,11,1),
    'tfidfvectorizer__max_df': np.arange(.08, .12, .005),
    'xgbclassifier__n_estimators': [100],
    'xgbclassifier__max_depth': np.arange(2,7,1)
}

kf = StratifiedKFold(n_splits=3)

pl = make_pipeline(
    TfidfVectorizer(),
    XGBClassifier()
)

grid = GridSearchCV(pl, param_grid, cv=kf, scoring='roc_auc')\
.fit(x, y)

clf = grid.best_estimator_

clf

ValueError: multiclass format is not supported